In [1]:
import pandas as pd
import numpy as np

In [19]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import MiniBatchKMeans

In [20]:
def reduce_img_colours(img_path, compression = 1, num_colours = 5):
    
    img = Image.open(img_path)
    
    new_size = (np.array(img.size) * compression).astype(int)
    
    img_array = np.array(img.resize(new_size))
    
    pixels = img_array.reshape((-1, 3))
    
    kmeans = MiniBatchKMeans(num_colours).fit(pixels)
    
    new_pixels = kmeans.cluster_centers_[kmeans.labels_]
    
    new_img = new_pixels.reshape((new_size[1], new_size[0], 3))
    
    print(f'# Old colours: {len(np.unique(pixels, axis = 0))}')
    print(f'# New colours: {num_colours}')
    
    return new_img

In [26]:
# https://github.com/google-research-datasets/conceptual-captions/blob/master/LICENSE
# https://ai.google.com/research/ConceptualCaptions/download
data = pd.read_csv('../raw_data/Validation_GCC-1.1.0-Validation.tsv', sep = '\t', header = None)

In [27]:
data.columns = ['caption', 'url']

In [174]:
import requests
from PIL import Image
import PIL
import io
import torch
import clip

def get_pil_image(url):
    
    try:
        response = requests.get(url)
        if response.status_code == 200:
                return Image.open(io.BytesIO(response.content))
    
    except requests.exceptions.RequestException:
        return None
    except PIL.UnidentifiedImageError:
        return None
#     except RuntimeError:
#         return None


def square_image(pil_img):
    
    short_side = min(pil_img.size)
    
    img_width, img_height = pil_img.size
    return pil_img.crop(((img_width - short_side) // 2,
                         (img_height - short_side) // 2,
                         (img_width + short_side) // 2,
                         (img_height + short_side) // 2))

def simplify_image(pil_img, num_colors = 8):

    img = pil_img.quantize(colors=num_colors, method=2)
    
    sq_img = square_image(img).resize((256, 256), Image.LANCZOS)
    
    return sq_img
    
def save_img(pil_img, caption, destination_folder):
    
    file_name = f"{destination_folder}/{caption.replace('.', '')}.png"
    
    #print('saving to ', file_name)
    
    pil_img.save(file_name)


model, preprocess = clip.load("ViT-B/32", device=device)


def clip_encode_image(pil_img):
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    image = preprocess(pil_img).unsqueeze(0).to(device)

    with torch.no_grad():
        image_features = model.encode_image(image)

    return image_features



In [175]:
from tqdm import tqdm

#features_list = []
#image_path_list = []

for i, row in tqdm(data[1100:2000].iterrows()):
    
    url, caption = row['url'], row['caption']
    
    #print(url)

    img = get_pil_image(url)
    
    if img:

        features_list.append(clip_encode_image(img)[0])

        simple_img = simplify_image(img, num_colors = 256)

        save_img(simple_img, caption, destination_folder = '../raw_data/images')
        
        image_path_list.append(f"{caption.replace('.', '')}.png")
    if len(features_list) % 10 == 0:
        print(len(features_list))
        
        

#image = get_simplified_image(url, caption, destination_folder = '../raw_data/images', num_colors = 256)

7it [00:09,  1.36s/it]

160


19it [01:01,  6.98s/it]

170


30it [01:18,  1.35s/it]

...

570


523it [32:32,  1.91s/it]

580


523it [52:15,  5.99s/it]


KeyboardInterrupt: 

In [176]:
image_embeddings = np.stack(features_list)

In [177]:
image_embeddings.shape

X_image_embeddings = torch.from_numpy(image_embeddings)
X_image_embeddings

tensor([[ 0.0410, -0.1044, -0.3145,  ...,  0.4066,  0.1564,  0.1414],
        [-0.0344,  0.1190,  0.2218,  ...,  0.6305, -0.3967, -0.0071],
        [-0.2324,  0.0697,  0.0915,  ...,  0.3114,  0.0565, -0.3759],
        ...,
        [-0.3113, -0.0241, -0.1355,  ...,  0.7593, -0.2546, -0.2554],
        [ 0.0336, -0.1806, -0.0556,  ...,  0.2804,  0.2809,  0.1955],
        [-0.6176, -0.1579, -0.5416,  ...,  0.7598,  0.0930,  0.0875]])

In [163]:
# mnist = pymde.datasets.MNIST()
# mnist.data

In [178]:
import pymde

mde = pymde.preserve_neighbors(X_image_embeddings, embedding_dim=2, verbose=True)
embedding = mde.embed(verbose=True)

Jul 02 01:25:39 AM: Computing 5-nearest neighbors, with max_distance=None
Jul 02 01:25:39 AM: Exact nearest neighbors by brute force 
Jul 02 01:25:39 AM: Computing quadratic initialization.
Jul 02 01:25:39 AM: Fitting a centered embedding into R^2, for a graph with 580 items and 4213 edges.
Jul 02 01:25:39 AM: `embed` method parameters: eps=1.0e-05, max_iter=300, memory_size=10
Jul 02 01:25:39 AM: iteration 000 | distortion 0.529098 | residual norm 0.0905655 | step length 0.51596 | percent change 0.137199
Jul 02 01:25:39 AM: iteration 030 | distortion 0.280032 | residual norm 0.00242741 | step length 1 | percent change 0.989663
Jul 02 01:25:39 AM: iteration 060 | distortion 0.277166 | residual norm 0.00110947 | step length 1 | percent change 0.894307
Jul 02 01:25:40 AM: iteration 090 | distortion 0.275297 | residual norm 0.000927837 | step length 1 | percent change 0.513152
Jul 02 01:25:40 AM: iteration 120 | distortion 0.274814 | residual norm 0.000516245 | step length 1 | percent cha

In [179]:
image_path_list

['author : a life in photography -- in pictures.png',
 'the player staring intently at a computer screen .png',
 'the - bedroom stone cottage can sleep people.png',
 'party in the park under cherry blossoms.png',
 'a man holds what is believed to be some of the debris that caused damage to vehicles monday afternoon after airliner returned to airport following problems after take off .png',
 "where 's the best place to show off your nails ? right in front of the castle , of course !.png",
 'that combines elements of a simple vegetable and dish.png',
 'transformers : till all are issue # 4b.png',
 'illustration of a little girl taking a bath.png',
 'tv police procedural is filming on the street this week .png',
 'the new terminal on island with quiet sea and setting sun.png',
 'what makeup to wear to a job interview.png',
 'the dentist drill the tooth with a turbine.png',
 'pair of new bright orange modern sneakers isolated on a white background.png',
 'ask industry to do in our family r

In [191]:

Y_coords = embedding.numpy()[:, 1]

In [219]:
def embedding_to_coords(embedding, axis):
    
    if axis == 'X':
        col = 0
    elif axis == 'Y':
        col = 1
    else:
        print("coord must be X or Y")
    
    coords = embedding.numpy()[:, col]

    from_one = coords - min(coords)

    return [int(coord) for coord in (from_one / max(from_one) * 1000)]

In [221]:
X_coords = embedding_to_coords(embedding, 'X')
type(X_coords[0])

int

In [222]:
import json

X_coords = embedding_to_coords(embedding, 'X')
Y_coords = embedding_to_coords(embedding, 'Y')

map_json = {
    "X_coords": X_coords,
    "Y_coords": Y_coords,
    "image_paths": image_path_list
}
map_json['image_paths'][0]
with open('../raw_data/latent_space_map.json', 'w') as f:
    json.dump(map_json, f)
